# Perfil de comércio

## Criar directorias de apoio 

In [1]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

## Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [2]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = 
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


## Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


## Inicializar a API e transferir os códigos usados

In [3]:
import os
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    # get API Key or set to None    
    APIKEY = config['comtrade'].get('key', None)

comtrade.init(APIKEY, force_init=True)
comtrade.PERIOD_SECONDS=12


### Perfil de comércio

Escolher país e anos de interesse.

In [4]:
country_of_interest = comtrade.m49_angola
years_of_interest = comtrade.year_range(2019, 2021)
print(comtrade.COUNTRY_CODES[country_of_interest], years_of_interest)

Angola 2019,2020


### Global exports and imports

In [5]:
cols = ['reporterDesc','partnerDesc','flowDesc','primaryValueFormated']
global_trade = comtrade.get_global_stats(country_of_interest, years_of_interest)
global_trade[cols].sort_index()


reporterDesc partnerDesc flowDesc primaryValueFormated
refYear flowCode                                                       
2019    M              Angola       World   Import    13,961,640,425.76
        X              Angola       World   Export    35,432,454,858.83
2020    M              Angola       World   Import     9,337,925,708.64
        X              Angola       World   Export    22,134,505,473.38

## Principais parceiros comerciais
### Exportações

In [6]:
import pandas as pd
# show more lines
pd.set_option('display.max_rows', 500)

top_partners = comtrade.top_partners(country_of_interest,years_of_interest,flowCode='X', 
                                    global_stats=global_trade,rank_filter=5)
top_partners.sort_values(by=['refYear','flowCode','rank'], ascending=[True,True,True]).style.format({'perc': '{:,.2%}'.format})

## Principais produtos exportados

In [7]:
cols = ['reporterDesc','partnerDesc','refYear','rank','cmdCode','cmdDesc',
                    'flowCode','primaryValueFormated','primaryValue']
export_products = comtrade.top_commodities(country_of_interest,0,years_of_interest, flowCode='X',
                                           motCode=0, pco_cols=cols,rank_filter=5,
                                           global_stats=global_trade)


In [8]:
export_products[['cmdDesc','primaryValue','perc']].style.format({'perc': '{:,.2%}'.format})

TypeError: tuple indices must be integers or slices, not list

### Principais produtos por parceiros principais

In [10]:
top_partner_codes = top_partners.partnerCode.unique()
# create a csv string with the list of partner codes
top_partner_list = ','.join([str(x) for x in top_partner_codes])
top_partner_list


'156,699,724,784,620,764'

In [97]:
export_products, df = comtrade.top_commodities(country_of_interest,top_partner_list,years_of_interest, flowCode='X',
                                           motCode=0, 
                                           partner_first=True,
                                           global_stats=global_trade, 
                                           pco_cols=cols,rank_filter=3)


In [98]:
import pandas as pd
# show more lines
pd.set_option('display.max_rows', 2500)

export_products[['cmdDesc','primaryValue','perc']].reset_index().style.format({'perc': '{:,.2%}'.format})

,reporterDesc,partnerDesc,flowCode,refYear,rank,cmdDesc,primaryValue,perc
0,Angola,China,X,2019,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",21519816595.153000,60.73%
1,Angola,China,X,2019,2,Wood and articles of wood; wood charcoal,9692928.139000,0.03%
2,Angola,China,X,2019,3,"Salt; sulphur; earths, stone; plastering materials, lime and cement",4109571.063000,0.01%
3,Angola,China,X,2020,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",13177932927.525999,59.54%
4,Angola,China,X,2020,2,"Salt; sulphur; earths, stone; plastering materials, lime and cement",15210836.941000,0.07%
5,Angola,China,X,2020,3,"Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin",10492445.970000,0.05%
6,Angola,India,X,2019,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",3366899924.808000,9.50%
7,Angola,India,X,2019,2,"Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin",5093382.178000,0.01%
8,Angola,India,X,2019,3,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",3186585.665000,0.01%
9,Angola,India,X,2020,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",1602005705.744000,7.24%


In [115]:
show_cols=['reporterDesc','partnerDesc','refYear','flowCode','cmdCode','cmdDesc','primaryValueFormated','primaryValue']
sort_list=['refYear','flowCode','rank_year_flow_cmd']
df['sum_year_flow_cmd'] = df.groupby(['reporterDesc','refYear','flowCode','cmdCode'])['primaryValue'].transform('sum')  
df['rank_year_flow_cmd'] = df.groupby(['reporterDesc','refYear','flowCode'])['sum_year_flow_cmd'].rank(ascending=False,method='dense')
df[['reporterDesc','refYear','flowCode','partnerDesc','cmdCode','cmdDesc','primaryValue','sum_year_flow_cmd','rank_year_flow_cmd']].sort_values(sort_list).head(50)


,reporterDesc,refYear,flowCode,partnerDesc,cmdCode,cmdDesc,primaryValue,sum_year_flow_cmd,rank_year_flow_cmd
550,Angola,2019,X,Spain,27,"Mineral fuels, mineral oils and products of th...",1.230509e+09,2.791543e+10,1.0
551,Angola,2019,X,United Arab Emirates,27,"Mineral fuels, mineral oils and products of th...",6.750449e+07,2.791543e+10,1.0
593,Angola,2019,X,Thailand,27,"Mineral fuels, mineral oils and products of th...",6.311109e+08,2.791543e+10,1.0
600,Angola,2019,X,Portugal,27,"Mineral fuels, mineral oils and products of th...",1.099593e+09,2.791543e+10,1.0
609,Angola,2019,X,India,27,"Mineral fuels, mineral oils and products of th...",3.366900e+09,2.791543e+10,1.0
644,Angola,2019,X,China,27,"Mineral fuels, mineral oils and products of th...",2.151982e+10,2.791543e+10,1.0
15,Angola,2019,X,India,71,"Natural, cultured pearls; precious, semi-preci...",5.093382e+06,1.060714e+09,2.0
475,Angola,2019,X,United Arab Emirates,71,"Natural, cultured pearls; precious, semi-preci...",1.055602e+09,1.060714e+09,2.0
689,Angola,2019,X,Portugal,71,"Natural, cultured pearls; precious, semi-preci...",1.858916e+04,1.060714e+09,2.0
111,Angola,2019,X,Spain,03,"Fish and crustaceans, molluscs and other aquat...",1.778729e+07,1.778729e+07,3.0


In [83]:
sort_list = ['reporterDesc','refYear','flowCode','cmdCode']
sort_order = [True,True,True,True]
groupby_list = ['reporterDesc','cmdCode','flowCode','refYear']

sort_list_cmd = ['refYear','flowCode','cmdCode']
sort_order_cmd = [True,True,True]
groupby_list_cmd = ['refYear']

show_cols = ['cmdDesc','primaryValueFormated','rank']
pco = df.set_index(['reporterDesc','refYear','flowCode','cmdCode'])

pco['rank'] = pco.groupby(level=['reporterDesc','refYear','flowCode'])['primaryValue'].rank(method='dense',ascending=False)
pco[show_cols].sort_values([]).head(20)

cmdDesc  \
reporterDesc refYear flowCode cmdCode                                                      
Angola       2019    X        73                                  Iron or steel articles   
                              91                    Clocks and watches and parts thereof   
                              04       Dairy produce; birds' eggs; natural honey; edi...   
                              87       Vehicles; other than railway or tramway rollin...   
                              08       Fruit and nuts, edible; peel of citrus fruit o...   
                              97           Works of art; collectors' pieces and antiques   
                              08       Fruit and nuts, edible; peel of citrus fruit o...   
                              71       Natural, cultured pearls; precious, semi-preci...   
                              63       Textiles, made up articles; sets; worn clothin...   
                              87       Vehicles; other than railway or tramway rollin...   
                              04       Dairy produce; birds' eggs; natural honey; edi...   
                              67       Feathers and down, prepared; and articles made...   
                              87       Vehicles; other than railway or tramway rollin...   
                              02                              Meat and edible meat offal   
                              91                    Clocks and watches and parts thereof   
                              02                              Meat and edible meat offal   
                              64       Footwear; gaiters and the like; parts of such ...   
                              87       Vehicles; other than railway or tramway rollin...   
                              72                                          Iron and steel   
                              21                       Miscellaneous edible preparations   

                                      primaryValueFormated   rank  
reporterDesc refYear flowCode cmdCode                              
Angola       2019    X        73                525,154.43   30.0  
                              91                253,970.10   43.0  
                              04                    160.76  212.0  
                              87                631,750.21   28.0  
                              08              1,186,847.22   22.0  
                              97                 13,897.49   98.0  
                              08                 47,152.41   68.0  
                              71              5,093,382.18   11.0  
                              63                  1,945.07  145.0  
                              87                  9,012.64  107.0  
                              04                    549.30  182.0  
                              67                    922.69  166.0  
                              87                864,718.70   25.0  
                              02                    982.88  160.0  
                              91                  5,470.64  124.0  
                              02                 70,605.09   61.0  
                              64                  5,254.17  126.0  
                              87                    567.08  179.0  
                              72                802,565.37   27.0  
                              21                 10,692.51  104.0

## Obter os dados

In [4]:

import ipywidgets as widgets
from IPython.display import display
import comtrade

rank_filter = 5  # número de importações mais relevantes
years = "2021"
partnerCode = comtrade.m49_angola # 
flowCode="M"

# select year with widget from list range(2000,2022)

yearsWidget = widgets.SelectMultiple(
    options=list(range(2000,2022)),
    description='Ano:',
    disabled=False,
)

countries = [("China", comtrade.m49_china),("Macau",comtrade.m49_macau) , ("Hong Kong", comtrade.m49_hong_kong)] + comtrade.PLP_TUPLES_REVERSE
# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
reporterCodeWidget = widgets.Dropdown(
    options=countries,
    description='Parceiro A:',
    disabled=False,
)

# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
partnerCodeWidget = widgets.Dropdown(
    options=countries,
    description='Parceiro B:',
    disabled=False,
)

# select flowCode widget from list M=Importações, X=Exportações
flowCodeWidget = widgets.Dropdown(
    options=[("Importações", "M"), ("Exportações", "X"),("Ambos", "M,X")],
    value="M",
    description='Fluxo:',
    disabled=False, 
)



### Escolher ano, parceiro e fluxo (import/export) para análise 

Pode escolher-se mais que um ano.

In [8]:
from ipywidgets import VBox

items = [reporterCodeWidget, partnerCodeWidget,yearsWidget]
box = VBox(children=items)
# display vertically
box


In [9]:
yearsList = ",".join(list(map(str,yearsWidget.value)))
partnerA = reporterCodeWidget.value
partnerB = partnerCodeWidget.value
print(yearsList, partnerA, partnerB)

 156 156


## Análise de simetrias nos dados

In [7]:
cols = ['reporterDesc','partnerDesc','refYear','flowDesc','primaryValueFormated','primaryValue']

In [8]:
flowCode = 'M'
df_import_rep_a = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode='TOTAL',
                     reporterCode=partnerA,
                     partnerCode=partnerB,
                     partner2Code=0,
                     period=yearsList,
                     motCode = 0,
                     timeout=30,
                     echo_url=True
                     )
#df_import_rep_a.info()

https://comtradeapi.un.org/data/v1/get//C/A/HS?reporterCode=156&period=&partnerCode=156&partner2Code=0&cmdCode=TOTAL&flowCode=M&customsCode=C00&subscription-key=HIDDEN


In [9]:
print(f"Importações de produtos de {comtrade.COUNTRY_CODES.get(partnerA)} por {comtrade.COUNTRY_CODES.get(partnerB)} reportados por {comtrade.COUNTRY_CODES.get(partnerA)}")
df_import_rep_a[cols]

Importações de produtos de China por China reportados por China


,reporterDesc,partnerDesc,refYear,flowDesc,primaryValueFormated,primaryValue
0,China,China,2022,Import,"122,883,424,107.00",1.228834e+11
1,China,China,2021,Import,"156,820,006,224.00",1.568200e+11
2,China,China,2020,Import,"125,264,219,826.00",1.252642e+11
3,China,China,2019,Import,"129,792,198,042.00",1.297922e+11
4,China,China,2018,Import,"146,221,550,788.00",1.462216e+11
5,China,China,2017,Import,"132,369,371,926.00",1.323694e+11
6,China,China,2016,Import,"128,059,540,969.00",1.280595e+11
7,China,China,2015,Import,"143,369,308,743.00",1.433693e+11
8,China,China,2014,Import,"144,880,436,618.00",1.448804e+11
9,China,China,2013,Import,"157,540,887,639.00",1.575409e+11


In [10]:
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode='TOTAL',
                     reporterCode=partnerA,
                     partnerCode=None,
                     partner2Code=0,
                     period=yearsList,
                     motCode = 0,
                     timeout=30,
                     echo_url=True
                     )

https://comtradeapi.un.org/data/v1/get//C/A/HS?reporterCode=156&period=&partner2Code=0&cmdCode=TOTAL&flowCode=M&customsCode=C00&subscription-key=HIDDEN


/Users/jrc/develop/cipf-comtrade/comtrade.py:276: UserWarning: Server returned HTTP Status: 429
  warnings.warn(f"Server returned HTTP Status: {str(resp.status_code)}",)
/Users/jrc/develop/cipf-comtrade/comtrade.py:279: UserWarning: Server returned error: Rate limit is exceeded. Try again in 9 seconds.
  warnings.warn(f"Server returned error: {message}")


In [11]:
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>100000000][cols].sort_values(by=['refYear','primaryValue'], ascending=False).head(100)

TypeError: 'NoneType' object is not subscriptable

In [91]:
# invertemos

print(yearsList, partnerA, partnerB)
print(f"Exportações de produtos de {comtrade.COUNTRY_CODES.get(partnerB)} para {comtrade.COUNTRY_CODES.get(partnerA)} reportados por {comtrade.COUNTRY_CODES.get(partnerB)}")

flowCode = 'X'
df_import_rep_b = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode='TOTAL',
                     reporterCode=partnerB,
                     partnerCode=partnerA,
                     partner2Code=None,
                     period=yearsList,
                     motCode = 0,
                     timeout=30,
                     echo_url=True
                     )
# df_import_rep_b.info()

2018,2019,2020,2021 24 156
Exportações de produtos de China para Angola reportados por China
https://comtradeapi.un.org/data/v1/get//C/A/HS?reporterCode=156&period=2018%2C2019%2C2020%2C2021&partnerCode=24&cmdCode=TOTAL&flowCode=X&customsCode=C00&subscription-key=HIDDEN


In [92]:
df_import_rep_a[cols]

,reporterDesc,partnerDesc,refYear,flowDesc,primaryValueFormated,primaryValue
0,Angola,China,2018,Import,"2,331,088,265.89",2.331088e+09
4,Angola,China,2019,Import,"2,026,243,881.71",2.026244e+09
11,Angola,China,2020,Import,"1,434,674,061.72",1.434674e+09
15,Angola,China,2021,Import,"1,688,904,641.34",1.688905e+09


In [93]:
df_import_rep_b[cols]

,reporterDesc,partnerDesc,refYear,flowDesc,primaryValueFormated,primaryValue
0,China,Angola,2018,Export,"2,253,137,107.00",2.253137e+09
1,China,Angola,2019,Export,"2,055,727,417.00",2.055727e+09
2,China,Angola,2020,Export,"1,748,092,204.00",1.748092e+09
3,China,Angola,2021,Export,"2,491,545,681.00",2.491546e+09
